In [1]:
import os 

In [2]:
%pwd

'd:\\DataScience\\DS\\Neural Networks\\tensor_flow\\end-to-end-chestClassificationDL\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\DataScience\\DS\\Neural Networks\\tensor_flow\\end-to-end-chestClassificationDL'

In [5]:
# entity part
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path

In [6]:
#  Update the configuration manager in src config
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories


In [7]:
class CofigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request 
import zipfile
import gdown 
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [9]:
# Update the cmponents 
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self) -> str:
        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extract the zip file into the data directory 
        Function returns None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# Update the pipeline 

try:
    config = CofigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
        raise e

[2025-08-26 21:42:17,655: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-26 21:42:17,665: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-26 21:42:17,672: INFO: common: created directory at: artifacts]
[2025-08-26 21:42:17,672: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-26 21:42:17,686: INFO: 3839920159: Downloading data from https://drive.google.com/file/d/1k06l3lkH9G_uEn9UWD4OMsFeFG1nGqr6/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1k06l3lkH9G_uEn9UWD4OMsFeFG1nGqr6
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1k06l3lkH9G_uEn9UWD4OMsFeFG1nGqr6&confirm=t&uuid=cf25d326-b767-495d-a73f-17ff0b68c6fe
To: d:\DataScience\DS\Neural Networks\tensor_flow\end-to-end-chestClassificationDL\artifacts\data_ingestion\data.zip
100%|██████████| 124M/124M [10:49<00:00, 191kB/s]    

[2025-08-26 21:53:14,306: INFO: 3839920159: Downloaded data from https://drive.google.com/file/d/1k06l3lkH9G_uEn9UWD4OMsFeFG1nGqr6/view?usp=sharing into file artifacts/data_ingestion/data.zip]
